In [1]:
import pandas as pd
import numpy as np

text_file = "/home/coochie/vscode/DeepLearing/data/text_data/yogabar/August 2021 All Order Report (1).txt"
excel_file = "/home/coochie/vscode/DeepLearing/data/text_data/yogabar/Mapper (1).xlsx"

In [4]:
orders = pd.read_csv(text_file, sep='\t')
mapper  = pd.read_excel(excel_file)
print(orders.columns)

Index(['amazon-order-id', 'merchant-order-id', 'purchase-date',
       'last-updated-date', 'order-status', 'fulfillment-channel',
       'sales-channel', 'order-channel', 'url', 'ship-service-level',
       'product-name', 'sku', 'asin', 'number-of-items', 'item-status',
       'quantity', 'currency', 'item-price', 'item-tax', 'shipping-price',
       'shipping-tax', 'gift-wrap-price', 'gift-wrap-tax',
       'item-promotion-discount', 'ship-promotion-discount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'item-extensions-data', 'is-business-order', 'purchase-order-number',
       'price-designation', 'fulfilled-by', 'buyer-company-name',
       'buyer-cst-number', 'buyer-vat-number', 'buyer-tax-registration-id',
       'buyer-tax-registration-country', 'buyer-tax-registration-type',
       'customized-url', 'customized-page', 'is-heavy-or-bulky',
       'is-replacement-order', 'is-exchange-order', 'original-order-id',
       'is-amazo

/tmp/ipykernel_60804/505011847.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv(text_file, sep='\t')


In [3]:
orders

,amazon-order-id,merchant-order-id,purchase-date,last-updated-date,order-status,fulfillment-channel,sales-channel,order-channel,url,ship-service-level,...,customized-page,is-heavy-or-bulky,is-replacement-order,is-exchange-order,original-order-id,is-amazon-invoiced,vat-exclusive-item-price,vat-exclusive-shipping-price,vat-exclusive-giftwrap-price,is-iba
0,406-5939791-5186746,406-5939791-5186746,2021-08-31T18:29:57+00:00,2021-09-01T08:16:46+00:00,Shipped,Amazon,Amazon.in,NaN,NaN,Expedited,...,NaN,False,False,False,NaN,False,400.0,0.0,NaN,False
1,402-9449649-9730763,402-9449649-9730763,2021-08-31T18:28:38+00:00,2021-09-06T12:34:02+00:00,Cancelled,Amazon,Amazon.in,NaN,http://www.amazon.in,Standard,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False
2,406-4315875-5867532,406-4315875-5867532,2021-08-31T18:26:58+00:00,2021-08-31T22:39:16+00:00,Cancelled,Amazon,Amazon.in,NaN,NaN,Expedited,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False
3,171-3308347-6453127,171-3308347-6453127,2021-08-31T18:25:33+00:00,2021-08-31T18:26:28+00:00,Cancelled,Amazon,Amazon.in,NaN,NaN,Expedited,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False
4,404-4597513-6310729,404-4597513-6310729,2021-08-31T18:24:20+00:00,2021-09-01T05:36:39+00:00,Shipped,Amazon,Amazon.in,NaN,NaN,Expedited,...,NaN,False,False,False,NaN,False,400.0,0.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33188,408-7768562-2088345,408-7768562-2088345,2021-07-31T18:34:16+00:00,2021-08-10T15:11:51+00:00,Cancelled,Amazon,Amazon.in,NaN,www.amazon.in,Standard,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False
33189,403-5192636-4530763,403-5192636-4530763,2021-07-31T18:33:29+00:00,2021-08-01T10:38:26+00:00,Cancelled,Amazon,Amazon.in,NaN,www.amazon.in,Standard,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False
33190,403-7070782-3914710,403-7070782-3914710,2021-07-31T18:32:16+00:00,2021-08-01T10:38:37+00:00,Cancelled,Amazon,Amazon.in,NaN,www.amazon.in,Standard,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False
33191,403-9730198-4038726,403-9730198-4038726,2021-07-31T18:31:30+00:00,2021-08-10T00:32:03+00:00,Cancelled,Amazon,Amazon.in,NaN,www.amazon.in,Standard,...,NaN,False,False,False,NaN,False,0.0,0.0,NaN,False


In [66]:
def clean_orders(data):
    data['purchase-date'] = pd.to_datetime(data['purchase-date'], format='%Y-%m-%dT%H:%M:%S%z')
    data['date'] = pd.to_datetime(data['purchase-date'], format='%Y-%m-%dT%H:%M:%S%z').dt.date
    data['week'] = 'week-' + data['purchase-date'].dt.isocalendar().week.astype(str)
    return data

order_data = clean_orders(orders)

In [67]:
# 1 st question

df_product_daily_sales = order_data.groupby(['product-name', 'date'])['item-price'].sum().reset_index()
df_product_daily_sales = df_product_daily_sales.sort_values(by='date')
df_product_daily_sales

,product-name,date,item-price
1679,Yogabar Quick Cooking Oats 900g | Premium Oats...,2021-07-31,558.0
1385,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,2021-07-31,1600.0
848,Yogabar Chocolate Chunk Multigrain-Energy Bars...,2021-07-31,0.0
629,Yogabar Breakfast Protein Variety (Almond Coco...,2021-07-31,1160.0
2087,Yogabar Wholegrain Breakfast Muesli - Dark Cho...,2021-07-31,0.0
...,...,...,...
284,"Yogabar 20G Protein Almond Bar - Pack of 12, K...",2021-08-31,2400.0
2214,Yogabar Wholegrain Breakfast Muesli Combo | Fr...,2021-08-31,31996.0
800,Yogabar Breakfast Protein-Bar Blueberry Pie - ...,2021-08-31,500.0
227,Yogabar 20 gram Protein Bar Chocolate Brownie ...,2021-08-31,8985.0


In [68]:
# 2nd question

# Top 10 products for the month with week on week sales trend views
df_top_10_products = order_data.groupby('product-name')['item-price'].sum().reset_index()
df_top_10_products = df_top_10_products.nlargest(10, 'item-price')
df_top_10_products

,product-name,item-price
88,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0
17,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,909935.0
145,Yogabar Wholegrain Breakfast Muesli Combo | Fr...,837235.0
57,Yogabar Crunchy Peanut Butter 1kg | Dark Choco...,792967.0
127,"Yogabar Super Muesli, No Added or Hidden Sugar...",696882.0
14,Yogabar 20 Gram Protein Bar Variety Box - 6 x ...,579157.4
7,Yogabar 100% Rolled Oats 1.2 kg | Premium Gold...,498800.0
77,Yogabar Dark Chocolate Peanut Butter| Creamy &...,462617.0
9,Yogabar 100% Rolled Oats 1.2 kg | Premium Gold...,376400.0
16,Yogabar 20 gram Protein Bar Chocolate Brownie ...,369811.0


In [69]:
# Create a week on week sales trend view
df_top_10_products_weekly_sales = order_data.groupby(['product-name','week' ])['item-price'].sum().reset_index()
df_top_10_products_weekly_sales = df_top_10_products.merge(df_top_10_products_weekly_sales, on='product-name')
df_top_10_products_weekly_sales

,product-name,item-price_x,week,item-price_y
0,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0,week-30,39960.0
1,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0,week-31,275410.0
2,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0,week-32,277600.0
3,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0,week-33,308670.0
4,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0,week-34,269185.0
5,Yogabar Multigrain Energy Bars 380Gm Pack (38G...,1245545.0,week-35,74720.0
6,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,909935.0,week-30,17970.0
7,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,909935.0,week-31,182029.0
8,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,909935.0,week-32,219192.0
9,Yogabar 20 gram Protein Bar Variety Box - 6 x ...,909935.0,week-33,233499.0


In [70]:
# 3rd question


# Category Wise day on day sales and week on week sales
df_category_daily_sales = order_data.merge(mapper, on='sku')[['Category', 'date', 'item-price']].groupby(['Category', 'date'])['item-price'].sum().reset_index()
df_category_daily_sales = df_category_daily_sales.sort_values(by='date')
df_category_daily_sales

,Category,date,item-price
0,Breakfast Bar,2021-07-31,1450.0
95,Muesli,2021-07-31,3400.0
159,Peanut Butter,2021-07-31,229.0
191,Protein Bar,2021-07-31,3495.0
32,Combo,2021-07-31,1299.0
...,...,...,...
126,Muesli,2021-08-31,87831.0
190,Peanut Butter,2021-08-31,68451.0
94,Energy Bar,2021-08-31,8580.0
63,Combo,2021-08-31,6984.0


In [71]:
df_category_weekly_sales = order_data.merge(mapper, on='sku')[['Category', 'purchase-date', 'item-price' ,'week']].groupby(['Category', 'week'])['item-price'].sum().reset_index()
df_category_weekly_sales

,Category,week,item-price
0,Breakfast Bar,week-30,20784.0
1,Breakfast Bar,week-31,118366.0
2,Breakfast Bar,week-32,119989.0
3,Breakfast Bar,week-33,104582.0
4,Breakfast Bar,week-34,89804.0
5,Breakfast Bar,week-35,25844.0
6,Combo,week-30,13294.0
7,Combo,week-31,62972.6
8,Combo,week-32,79341.0
9,Combo,week-33,68480.0


In [76]:
# 4th question


# Product wise discount % on MRP Sales for the month
df_product_discount = order_data.merge(mapper, on='sku')[['product-name', 'item-price', 'MRP']].groupby('product-name').apply(lambda x: ((x['MRP'] - x['item-price']) / x['MRP']).mean() * 100).round(2).reset_index()
df_product_discount.columns = ['product-name', 'discount %']
df_product_discount

/tmp/ipykernel_37079/1433312969.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_product_discount = order_data.merge(mapper, on='sku')[['product-name', 'item-price', 'MRP']].groupby('product-name').apply(lambda x: ((x['MRP'] - x['item-price']) / x['MRP']).mean() * 100).round(2).reset_index()


,product-name,discount %
0,Yogabar 100% Pure Peanut Butter | Creamy & Yum...,-44.38
1,Yogabar 100% Pure Peanut Butter | Creamy & Yum...,42.61
2,Yogabar 100% Pure Peanut Butter | Creamy & Yum...,38.56
3,Yogabar 100% Pure Peanut Butter | Creamy & Yum...,-29.72
4,Yogabar 100% Rolled Oats 1.2 kg (Buy 1 Get 1 F...,20.88
...,...,...
136,Yogabar Wholegrain Breakfast Muesli | Almond Q...,23.47
137,Yogabar Wholegrain Breakfast Muesli | Dark Cho...,19.64
138,Yogabar Wholegrain Breakfast Muesli | Dark Cho...,-16.00
139,Yogabar Wholegrain Breakfast Muesli |Fruits Nu...,-9.09
